Need to run `export PYSPARK_PYTHON=python3.4` in commandline to use python3.4 as default python for Spark Cluster. Spark cluster already has `numpy` and `scipy`.

## 5/1/19

### *m4.xlarge* instance
The m4.xlarge instances have 4 vCPUs, 16GiB memory, and 32 GiB of EBS storage. The following information contain the architecture of the instance, the number of vCPUs, threads per vCPU/core, the processor, and the cache sizes:<br>
`Architecture:          x86_64
CPU(s):                4
On-line CPU(s) list:   0-3
Thread(s) per core:    2
Core(s) per socket:    2
Socket(s):             1
Model name:            Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              46080K`

### Testing sequential code on stand alone *m4.xlarge* instance using Python 3.5
12min and 54seconds to run sequential pipeline with *Genome-Pos-3T3-Unsorted_100_seqs.txt* and *Genome-Pos-3T3-Bot10_100_seqs.txt* input files. Both of these input file were constructed with 75 perfect matching reads and 25 that required ED calculation. About 12 of the ED sequencing reads come in the first 50 lines of each input file to split them up evenly.

### Testing spark code on stand alone *m4.xlarge* instance using Python 3.5 and Spark local mode
I used the *ml-latest/ratings.csv* file (709MB) provided on Canvas.<br>
To run the `P21_spark.py` code on the large version of the *ratings.csv* using 2, 3 or 4 cores, I modified my `P21_spark.py` code by including a `[c]` (where $c=2,3,4$) in the code at `SparkConf().setMaster('local[c]')`. I saved the modified code as `P26_spark.py`. 

I used the following command to run my tests:<br>
`spark-submit P26_spark.py "5.0"`.

The table below lists the number of cores used in local mode, the elapsed time, and the speed-up when I use 2, 3 or 4 cores.

|Number of cores| Elapsed time |Speed-up|
|---------------|--------------|--------|
|1 | 57.33 sec | -|
|2 | 34.74 sec |1.65|
|3 | 33.51 sec |1.71|
|4 | 34.25 sec |1.67|

As expected, we see an improvement in the performance when we increase the number of cores we use because with 2, 3 or 4 cores we get speed-ups between 1.65 and 1.71. The speed-up does not increase when using more than 2 cores because AWS only assigns 2 physical cores to m4xlarge instances. When looking at the speed-up between 1 and 2 cores, we do not observe a linear speed-up most likely due to overhead costs for running Spark.
